In [3]:
import pandas as pd
import numpy as np

ratings = pd.read_csv('/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/BX-Book-Ratings.csv', encoding='latin-1')
items = pd.read_csv('/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/BX-Books.csv', encoding='latin-1')
users = pd.read_csv('/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/BX-Users.csv', encoding='latin-1')


##### rating = k-fold 로 3번씩 split

In [38]:
rating = ratings[ratings['Book-Rating'] != 0 ]
rating_0 = ratings[ratings['Book-Rating'] == 0 ]
rating = rating[['User-ID', 'ISBN', 'Book-Rating']].reset_index(drop = True)

In [109]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=3)

X = rating[['User-ID', 'ISBN', 'Book-Rating']]
y = rating['Book-Rating']

X_train_folds = []
X_test_folds = []
y_train_folds = []
y_test_folds = []

for train_index, test_index in kf.split(X):
    # Print the indices of the current fold
    print("TRAIN:", train_index, "TEST:", test_index)
   
    # Append the train-test pairs to the respective lists
    X_train_folds.append(X.iloc[train_index])
    X_test_folds.append(X.iloc[test_index])
    y_train_folds.append(y.iloc[train_index])
    y_test_folds.append(y.iloc[test_index])

    

TRAIN: [11841 11842 11843 ... 35519 35520 35521] TEST: [    0     1     2 ... 11838 11839 11840]
TRAIN: [    0     1     2 ... 35519 35520 35521] TEST: [11841 11842 11843 ... 23679 23680 23681]
TRAIN: [    0     1     2 ... 23679 23680 23681] TEST: [23682 23683 23684 ... 35519 35520 35521]


In [ ]:
X_test_folds[0]['Book-Rating'] = 0 
X_test_folds[1]['Book-Rating'] = 0 
X_test_folds[2]['Book-Rating'] = 0 

In [9]:
import pandas as pd
test_2_answer= pd.read_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/test_2_answer.csv")

In [11]:
ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276762,3404122879,0
1,276762,3404182928,0
2,276762,3426690179,0
3,276762,3442424216,0
4,276762,3442425573,0


In [13]:
test_2_answer.head(5)

,Unnamed: 0,Book-Rating
0,11841,7
1,11842,7
2,11843,7
3,11844,6
4,11845,9


In [89]:
train_1 = pd.concat([rating_0, X_train_folds[0]])
test_1 = X_test_folds[0]
test_1_answer = pd.DataFrame(y_test_folds[0])

In [92]:
train_2 = pd.concat([rating_0, X_train_folds[1]])
test_2 = X_test_folds[1]
test_2_answer = pd.DataFrame(y_test_folds[1])

In [93]:
train_3 = pd.concat([rating_0, X_train_folds[2]])
test_3 = X_test_folds[2]
test_3_answer = pd.DataFrame(y_test_folds[2])

In [102]:
train_1.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/train_1.csv")
test_1.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/test_1.csv")
test_1_answer.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/test_1_answer.csv")
train_2.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/train_2.csv")
test_2.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/test_2.csv")
test_2_answer.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/test_2_answer.csv")
train_3.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/train_3.csv")
test_3.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/test_3.csv")
test_3_answer.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/test_3_answer.csv")

##### items / users 이상치 처리 

In [4]:
## 'Year-Of-Publication' 이상치 == 1378 & 2026
# 1378 = rating 1 개 -> 삭제
# 2026 = rating 1개 아니고 5개라 Book-Author : Kathy Lette가 2002년에 publication 있어서, 2006으로 대체 
items.describe()

items.drop(items[items['Year-Of-Publication'] == 1378].index, inplace = True)
items.loc[items['Year-Of-Publication'] == 2026, 'Year-Of-Publication'] = 2006

## 'Book-Title' 이상치 == name에 "#NAME"
fault_name_idx = items[items['Book-Title'].str.contains("#NAME")].index
items.drop(fault_name_idx, inplace = True)

# ISBN 기준 중복행 제거 
#(ISBN  == '051513628X' 일때 Book title 이 'Key of Light (Key Trilogy (Paperback))',Key of Light' 각각이라 2개 표시 되있는데, 그냥 긴제목으로 남기고 삭제함 )

items.drop_duplicates(inplace = True)
items.drop(items.query("ISBN == '051513628X'").index[1], inplace = True)

# 기본적인 진행 위에까지 했을때 남은 행 44129 

In [5]:
items

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1,1551103982,The Cycling Adventures of Coconut Head: A Nort...,Ted Schredd,1900,Graphic Arts Center Pub Co,http://images.amazon.com/images/P/1551103982.0...,http://images.amazon.com/images/P/1551103982.0...,http://images.amazon.com/images/P/1551103982.0...
2,013022393X,All the Best Salads and Salad Dressings,J. Warner,1911,Prentice Hall Direct,http://images.amazon.com/images/P/013022393X.0...,http://images.amazon.com/images/P/013022393X.0...,http://images.amazon.com/images/P/013022393X.0...
3,030758013X,Assorted Small Tray Puzzles,Golden,1920,Golden Books,http://images.amazon.com/images/P/030758013X.0...,http://images.amazon.com/images/P/030758013X.0...,http://images.amazon.com/images/P/030758013X.0...
4,082493069X,Grill and Barbecue Cooking,Ideals Publications Inc,1920,Ideals Publications,http://images.amazon.com/images/P/082493069X.0...,http://images.amazon.com/images/P/082493069X.0...,http://images.amazon.com/images/P/082493069X.0...
5,684717603,The GREAT GATSBY (Scribner Classic),F. Scott Fritzgerald,1920,Scribner Paper Fiction,http://images.amazon.com/images/P/0684717603.0...,http://images.amazon.com/images/P/0684717603.0...,http://images.amazon.com/images/P/0684717603.0...
...,...,...,...,...,...,...,...,...
44360,B00007MF56,"More, Now, Again: A Memoir of Addiction",Elizabeth Wurtzel,2001,Simon &amp Schuster,http://images.amazon.com/images/P/B00007MF56.0...,http://images.amazon.com/images/P/B00007MF56.0...,http://images.amazon.com/images/P/B00007MF56.0...
44361,B00007CWGV,Madonna,Andrew Morton,2001,St. Martin's Press,http://images.amazon.com/images/P/B00007CWGV.0...,http://images.amazon.com/images/P/B00007CWGV.0...,http://images.amazon.com/images/P/B00007CWGV.0...
44362,B000078UH8,The Hearing,John T. Lescroart,2001,E P Dutton,http://images.amazon.com/images/P/B000078UH8.0...,http://images.amazon.com/images/P/B000078UH8.0...,http://images.amazon.com/images/P/B000078UH8.0...
44363,B00006JO6O,The Last Precinct,Patricia Daniels Cornwell,2000,Putnam Pub Group,http://images.amazon.com/images/P/B00006JO6O.0...,http://images.amazon.com/images/P/B00006JO6O.0...,http://images.amazon.com/images/P/B00006JO6O.0...


In [6]:
# 소문자로 변경
def to_lower(series):
    return series.str.lower()

items['Book-Title'] = to_lower (items['Book-Title'])
items['Book-Author'] = to_lower (items['Book-Author'])
items['Publisher'] = to_lower (items['Publisher'])

##### Users

In [8]:
#Age 
#5세 미만, 100세 초과 제거 
del_age_index = users[(users['Age']<5 )|(users['Age']> 100)].index
users.drop(del_age_index, inplace = True)

# 10단위로 범주화하는 함수 정의
bins = range(0, 120, 10)  # 0부터 90까지 10단위로 범주 설정
users['Age_cate'] = pd.cut(users['Age'], bins=bins, labels=[f"{i}-{i+9}" for i in range(0, 110, 10)], right = False)
users['Age_cate'] = users['Age_cate'].apply(lambda x : '5-19' if (x == '0-9')or (x == '10-19') else ('90-100' if x == '90-99' or x == '100-109' else x))

In [9]:
# Location 
# location 시, 주, 도시로 분리 
temp = users["Location"].str.split(", ", expand = True)

In [10]:
# 5개로 분리되는 것들이 총 16개 있어서, 이들은 수작업으로 재분류함 temp[temp[4] == 'usa'] == 2191 번 한개임. 
temp[(temp[3] == 'united kingdom')|(temp[3] == 'usa')|(temp[3] == 'mexico')|(temp[3] == 'spain')|(temp[3] == 'georgia')| (temp[3] == 'canada')]


,0,1,2,3,4
236,coalville,leicestershire,england,united kingdom,None
703,tharston,norwich,england,united kingdom,None
1138,ermington,s. devon,england,united kingdom,None
1150,redmond,or,oregon,usa,None
1258,hauppauge,long island,new york,usa,None
1358,chicago,il,illinois,usa,None
1944,méxico,d.f.,disrito federal,mexico,None
2007,él médano,santa cruz de tenerife,canary islands,spain,None
2189,preston,lancashire,england,united kingdom,None
2191,merrick,acworth,new york,georgia,usa


In [11]:
condition_london = temp[temp[1]== 'london'].index
temp.loc[condition_london, 1] = 'england'

location_error_index= temp[(temp[3] == 'united kingdom')|(temp[3] == 'usa')|(temp[3] == 'mexico')|(temp[3] == 'spain')|(temp[3] == 'georgia')| (temp[3] == 'canada')].index 
temp.loc[location_error_index, 1] = temp.loc[location_error_index, 2]
temp.loc[location_error_index, 2]= temp.loc[location_error_index, 3]
#마지막 열까지 usa 있는것까지 대체 
temp.loc[temp[temp[4] == 'usa'].index, 2] = 'usa' 
temp.columns = ['city','state', 'country', 'error1', 'error2']

In [12]:
users = pd.concat([users, temp[['state', 'country']]], axis = 1)

In [13]:
users[users['country'].isna() == True]

,User-ID,Location,Age,Age_cate,state,country
360,20119,"manchester, new hampshire,",51.0,50-59,"new hampshire,",None
900,49771,"san antonio, texas,",35.0,30-39,"texas,",None
1486,81682,"bluffton, south carolina,",36.0,30-39,"south carolina,",None
1649,90198,"cornwall, new york,",34.0,30-39,"new york,",None
1795,98026,"yarmouth, nova scotia,",40.0,40-49,"nova scotia,",None
2146,117210,"st louis, missouri,",35.0,30-39,"missouri,",None
2789,151861,"sydney, new south wales,",40.0,40-49,"new south wales,",None
3602,195102,"lyon, rhône alpes,",26.0,20-29,"rhône alpes,",None
3770,204359,"brooklyn, new york,",40.0,40-49,"new york,",None


In [14]:
users[users['Location'].str.contains('rhône alpes')]

,User-ID,Location,Age,Age_cate,state,country
3198,172888,"valence, rhône alpes, france",57.0,50-59,rhône alpes,france
3602,195102,"lyon, rhône alpes,",26.0,20-29,"rhône alpes,",None
4596,248854,"lyon, rhône alpes, france",20.0,20-29,rhône alpes,france
4966,268945,"chambéry, rhône alpes, france",18.0,5-19,rhône alpes,france


In [15]:
#country 가 na 인 애들은 해당 state 이 있는 다른 사람들의 Country 찾아서 대체 시킴 
country_na_index = users[users['country'].isna() == True].index
#state에 , 붙어 있는거 다 없애는 전처리 
users.loc[country_na_index,'state'] = users.loc[country_na_index,'state'].str.replace(",", "")
#country 가 na 인 애들은 해당 state 이 있는 다른 사람들의 Country 찾아서 대체 시킴 
users.loc[country_na_index, 'country'] = users.loc[country_na_index, 'state'].apply(lambda x : 'usa' if x in ['new hampshire','texas', 'south carolina', 'new york', 'missouri'] else ('canada' if x == 'nova scotia'
                                                                                                                                              else ('australia' if x == 'new south wales'
                                                                                                                                                    else 'france')))

# state, country 모두 lower case 변경
users['state'] = to_lower(users['state'])
users['country'] = to_lower(users['country'])

In [16]:
#state 전처리
# dc : washington / dc 적절히 구분되어 있어서 그냥 냅둠
# ae : ae 가 Armed Forces Europe 의 약자긴 하다는데 따로 그렇게 쓴게 아닌듯 하여 냅둠
# nm : new mexico 약자인데 하나밖에 없어서 상관없을듯. 그냥 냅둠. 
# state : 'italy' : 대체하기가 애매해서 그냥 내둠 

# state : ga. / ga -> georgia / 'wa' -> wahsington / 'va' -> virginia / 'ca' -> california / 'tx' -> texas / 'co' -> colorado / '-' -> 'brussels'
state_keyword = ['ga.', 'ga', 'wa', 'va', 'ca', 'tx', 'co', '-']
state_kw_index = users[users['state'].isin(state_keyword)].index 

users.loc[state_kw_index, 'state'] = users.loc[state_kw_index, 'state'].apply(lambda x : 'georgia' if x in ['ga.', 'ga'] 
                                         else ('washington' if x == 'wa'
                                                else ('virginia' if x == 'va'
                                                      else ('california' if x== 'ca'
                                                        else ('texas' if x == 'tx'
                                                            else ('colorado' if x == 'co' else 'brussels'))))))

# del : 'my own little world',
users.drop(users[(users['Location'].str.contains('my own little world'))].index, inplace=  True)

# Location 에 'zaragoza' -> state : 'aragon' 
zaragoza_index = users[(users['Location'].str.contains('zaragoza'))].index 
users.loc[zaragoza_index, 'state'] = 'aragon'


In [17]:
# #country 전처리

# del : 'far away...',  'here and there'
users.drop(users[(users['Location'].str.contains('far away...'))].index, inplace= True)
users.drop(users[(users['Location'].str.contains('here and there'))].index, inplace= True)

#country : catalonia, 'catalunya'-> spain 
catalunya_index = users[(users['Location'].str.contains('catalunya'))| (users['Location'].str.contains('catalonia'))].index

users.loc[catalunya_index,'state'] = users.loc[catalunya_index,'state'].apply(lambda x : 'catalunya')
users.loc[catalunya_index,'country'] = users.loc[catalunya_index,'country'].apply(lambda x : 'spain')

#묶음 :  'usa', 'u.s.a.', 'united states', 'united state' / 'phillipines', 'philippines' / 
country_keyword = ['usa', 'u.s.a.', 'united states', 'united state','phillipines', 'philippines' ]
country_kw_index = users[users['country'].isin(country_keyword)].index 

users.loc[country_kw_index, 'country'] = users.loc[country_kw_index, 'country'].apply(lambda x : 'usa' if x in ['usa', 'u.s.a.', 'united states', 'united state'] 
                                         else ('philippines'))


users['country'].unique()

array(['portugal', 'germany', 'usa', 'spain', 'france', 'austria',
       'united kingdom', 'malaysia', 'taiwan', 'canada', 'iran', 'italy',
       'switzerland', 'romania', 'england', 'finland', 'philippines',
       'australia', 'india', 'qatar', 'japan', 'denmark', 'ireland',
       'belgium', 'chile', 'brazil', 'netherlands', 'papua new guinea',
       'new zealand', 'argentina', 'mexico', 'euskal herria', 'sweden',
       'china', 'hong kong', 'hungary', 'singapore', 'poland',
       'south africa', 'orense', 'kuwait', 'dominican republic', 'panama',
       'norway', 'turkey', 'costa rica', 'bulgaria', 'guatemala',
       'madagascar', 'czech republic', 'trinidad and tobago',
       'luxembourg'], dtype=object)

In [ ]:
##### 우선 어간 추출 전 데이터만 전처리 된거 => PR_books, PR_users

In [ ]:
items.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/PR_books.csv", index = False)
users.to_csv("/Users/soorimlee/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/23-02/Recommendation 복사본/team_project/data/PR_users.csv", index = False)

In [111]:
users

,User-ID,Location,Age
0,183,"porto, porto, portugal",27.0
1,242,"neuffen, baden-wuerttemberg, germany",37.0
2,254,"minneapolis, minnesota, usa",24.0
3,388,"nashville, tennessee, usa",35.0
4,392,"barcelona, catalunya, spain",44.0
...,...,...,...
5152,278356,"lakeland, tennessee, usa",57.0
5153,278563,"wien, wien, austria",37.0
5154,278668,"madrid, madrid, spain",48.0
5155,278843,"pismo beach, california, usa",28.0


In [1]:
import pandas as pd
import random
import re

In [2]:
# 데이터 로드 + 전처리
df_user = pd.read_csv('C:/RecoSys/Data/project/PR_users.csv', encoding='cp949')     #columns=[User-ID, Age_cate, state, country]
df_user['Age_cate'] = df_user['Age_cate'].apply(lambda x : '05-19' if x == '05월 19일' else x)  #엑셀 저장시 생기는 오류 보정

df_item = pd.read_csv('C:/RecoSys/Data/project/PR_books.csv', encoding='cp949')     #columns=[ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher]
df_item['Book-Title'] = df_item['Book-Title'].apply(lambda x : re.sub(r'[^a-z \s{0,}]','', x)) #특수문자 제거
df_item['Book-Author'] = df_item['Book-Author'].apply(lambda x : re.sub(r'[^a-z \s{0,}]','', str(x))) #특수문자 제거
df_item['Publisher'] = df_item['Publisher'].apply(lambda x : re.sub(r'[^a-z \s{0,}]','', x)) #특수문자 제거

In [6]:
book_id = {}
def convert_isbn(x) :
    if x in book_id :
        return book_id[x]
    else :
        book_id[x] = len(book_id)
        return book_id[x]
    
df_item.insert(1, 'Book-ID', df_item['ISBN'].apply(convert_isbn))
df_item.head()

,ISBN,Book-ID,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,1551103982,0,the cycling adventures of coconut head a north...,ted schredd,1900,graphic arts center pub co
1,013022393X,1,all the best salads and salad dressings,j warner,1911,prentice hall direct
2,030758013X,2,assorted small tray puzzles,golden,1920,golden books
3,082493069X,3,grill and barbecue cooking,ideals publications inc,1920,ideals publications
4,684717603,4,the great gatsby scribner classic,f scott fritzgerald,1920,scribner paper fiction


In [12]:
df_item.to_csv('C:/RecoSys/Data/project/PR_books2.csv', index=False)

In [7]:
df = pd.DataFrame()

for i in [1,2,3] :
    # 데이터 불러오기
    train = pd.read_csv('C:/RecoSys/Data/project/train_' + str(i) + '.csv')  #columns=[User-ID, ISBN, Book-Rating]

    # user & item과 병합
    train = pd.merge(df_user, train, on='User-ID', how='inner')
    train = pd.merge(df_item, train, on = 'ISBN', how='inner')
    train = train[['User-ID', 'Book-ID', 'ISBN', "Book-Title", 'Book-Author', 'Year-Of-Publication', 'Publisher', 
                   'Age_cate', 'state', 'country', 'Book-Rating']]

    # concat
    df = pd.concat([df, train], axis=0)

df = df.drop_duplicates()    # 중복 삭제
df = df.sample(frac=1)   # 행 셔플

df_0 = df[df['Book-Rating'] == 0].reset_index(drop=True)
df_no0 = df[df['Book-Rating'] != 0].reset_index(drop=True)

print(df.shape, df_0.shape, df_no0.shape)

(100061, 11) (64759, 11) (35302, 11)


In [8]:
df.head()

,User-ID,Book-ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age_cate,state,country,Book-Rating
6049,21364,3744,067162220X,where youll find me and other stories,ann beattie,1986,simon amp schuster,30-39,california,usa,0
34004,127429,17953,038531695X,the ghost,danielle steel,1997,dell,20-29,missouri,usa,0
16582,97099,9244,2010172213,andromaque,jean racine,1992,hachette education,20-29,massachusetts,usa,0
11641,229329,6833,2246417511,le pendule de foucault,umberto eco,1990,grasset,50-59,vaud,switzerland,10
55679,16795,28387,1551665743,mackenzies mountain,linda howard,2000,mira,40-49,maryland,usa,0


In [6]:
df[['User-ID', 'Book-Rating']].groupby('User-ID').sum().sort_values('Book-Rating', ascending=True)
df[['User-ID', 'Book-Rating']].groupby('User-ID').sum().describe()

,Book-Rating
count,5116.000000
mean,53.418100
std,88.679462
min,0.000000
25%,15.000000
50%,29.000000
75%,58.000000
max,2245.000000


In [9]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=3)

X = df_no0

train_folds = []
test_folds = []

for train_index, test_index in kf.split(X):
    # Print the indices of the current fold
    print("TRAIN:", train_index, "TEST:", test_index)
   
    # Append the train-test pairs to the respective lists
    train_folds.append(X.iloc[train_index])
    test_folds.append(X.iloc[test_index])

TRAIN: [11768 11769 11770 ... 35299 35300 35301] TEST: [    0     1     2 ... 11765 11766 11767]
TRAIN: [    0     1     2 ... 35299 35300 35301] TEST: [11768 11769 11770 ... 23532 23533 23534]
TRAIN: [    0     1     2 ... 23532 23533 23534] TEST: [23535 23536 23537 ... 35299 35300 35301]


In [10]:
for i in [0, 1, 2] :
    train = pd.concat([df_0, train_folds[i]])
    test = test_folds[i]

    train.to_csv('C:/RecoSys/Data/project/train' + str(i) + '.csv', index=False)
    test.to_csv('C:/RecoSys/Data/project/test' + str(i) + '.csv', index=False)

In [24]:
train['Age_cate'].unique()

array(['20-29', '30-39', '40-49', '60-69', '50-59', '05월 19일', '90-100',
       '70-79', '80-89'], dtype=object)